In [1]:
import src.config as config
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-02-26 19:00:00')


In [2]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-01-29 19:00:00'), to_date_=Timestamp('2022-02-27 19:00:00')
File 2022-01 was already in local storage
File 2022-02 was already in local storage


In [3]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 388.11it/s]


In [ ]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.FS_PROJECT_NAME,
    api_key_value=config.FS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FS_GROUP_NAME,
    version=config.FS_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

# insert data into feature group
feature_group.insert(ts_data, write_options={"wait_for_job": False})